<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1e842aaba2e3e6c42bc38346f0db5279b6a6d83cb6f19b7c0ee209c7217c9cf5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 09:45:43
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 19.07 K (0.13%)
Current PnL: -17.48 L (-11.75%)
CY Booked + Current PnL: -6.45 L (-4.34%)
-------------------
Total profit:  2.07 L
Total loss:  -19.54 L
-------------------
Total Booked + Current PnL: 20.53 L (16.87%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.58 L (58.97%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.86,57.0,H-LC,11.13,172194.0,16017.0,10073.0,-0.60,10.26,5.85,16.71,22.0,1.59,1.25,29.48,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,62.0,M-LC,4.25,218598.0,17118.0,10099.0,0.16,8.50,4.62,13.51,60.0,1.70,1.59,22.16,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.48,59.0,M-SC,4.71,88130.0,-12647.0,12744.0,0.41,-12.55,14.46,0.09,245.0,-0.99,0.64,15.78,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.00,72.0,H-MC,5.05,115158.0,14654.0,15984.0,-0.47,14.58,13.88,30.49,79.0,0.92,0.84,51.71,MH,ATH,METALS
41,ITC,452.00,-37.50,54.0,H-LC,2.90,202226.0,2088.0,18807.0,-0.05,1.04,9.30,10.44,4.0,0.11,1.47,6.66,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.88,43.0,H-MC,2.19,136142.0,354.0,42395.0,0.27,0.26,31.14,31.48,107.0,0.01,0.99,12.41,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.51,59.0,H-MC,1.76,246792.0,-939.0,75987.0,0.60,-0.38,30.79,30.30,92.0,-0.01,1.79,12.86,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.43,38.0,H-LC,5.81,300881.0,-14961.0,127634.0,0.38,-4.74,42.42,35.67,5.0,-0.12,2.19,8.46,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.42,267632.0,267.0,65195.0,0.00,0.10,24.36,24.48,48.0,0.00,1.95,55.13,XY24,BTT,FINANCE
31,HCLTECH,1943.91,0.22,46.0,H-LC,10.14,229079.0,-12845.0,76123.0,-0.55,-5.31,33.23,26.15,8.0,-0.17,1.67,8.25,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.02,225337.0,-8807.0,57438.0,0.03,-3.76,25.49,20.77,86.0,-0.15,1.64,2.90,X40N,NTT,BREWERIES
6,ASIANPAINT,4250.00,-12.01,51.0,H-LC,8.40,222033.0,-29735.0,151959.0,-0.91,-11.81,68.44,48.55,27.0,-0.20,1.61,19.43,X40,BTT,PAINTS


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.18,60.0,H-LC,1.08,239962.0,-6189.0,85402.0,-0.00,-2.51,35.59,32.18,16.0,-0.07,1.74,32.62,X200,ATH,IT
41,ITC,452.00,-37.50,54.0,H-LC,2.90,202226.0,2088.0,18807.0,-0.05,1.04,9.30,10.44,4.0,0.11,1.47,6.66,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.71,61.0,H-LC,3.95,284123.0,18697.0,38101.0,-0.28,7.04,13.41,21.40,11.0,0.49,2.07,14.64,XY25,NTT,FMCG
83,VBL,671.64,-16.43,38.0,H-LC,5.81,300881.0,-14961.0,127634.0,0.38,-4.74,42.42,35.67,5.0,-0.12,2.19,8.46,X40N,ATH,FMCG
56,RELIANCE,1533.00,-13.29,52.0,H-LC,5.85,217152.0,6786.0,21997.0,-0.22,3.23,10.13,13.68,37.0,0.31,1.58,20.28,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3872.97,65.0,M-SC,7.20,123829.0,-23539.0,75350.0,8.27,-15.97,60.85,35.16,236.0,-0.31,0.90,26.48,XY24,NTT,AUTO
75,TITAGARH,1548.0,2.50,72.0,H-SC,4.89,168096.0,-23919.0,102808.0,3.56,-12.46,61.16,41.08,158.0,-0.23,1.22,42.44,XY24,NTT,ENGINEERING
10,BAJAJHFL,152.0,-11.09,54.0,H-MC,6.10,187255.0,-13251.0,62019.0,1.92,-6.61,33.12,24.33,90.0,-0.21,1.36,7.98,X40N,BTT,FINANCE
66,SIS,528.0,2143.01,43.0,H-SC,5.37,90525.0,-20507.0,44113.0,1.92,-18.47,48.73,21.26,156.0,-0.46,0.66,22.31,OX40N,NTT,MISC
81,VAIBHAVGBL,521.0,54.27,62.0,H-SC,9.14,133860.0,-48915.0,159989.0,1.85,-26.76,119.52,60.77,125.0,-0.31,0.97,19.89,XR,NTT,JEWELLERY


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,680.0,-18.72,48.0,H-MC,6.57,218437.0,-8902.0,56968.0,-2.18,-3.92,26.08,21.14,106.0,-0.16,1.59,23.12,XY24,NTT,PAINTS
27,GREENPANEL,537.0,236.56,66.0,M-SC,4.74,150377.0,-32701.0,111685.0,-1.68,-17.86,74.27,43.14,230.0,-0.29,1.09,41.45,XY24,NTT,MISC
22,COFFEEDAY,80.0,-41.65,53.0,L-SC,45.55,85440.0,-28109.0,68164.0,-1.66,-24.75,79.78,35.27,268.0,-0.41,0.62,109.12,XR,NTT,HOTELS
46,LAOPALA,464.0,111.57,47.0,H-SC,2.29,96678.0,-34427.0,85676.0,-1.17,-26.26,88.62,39.09,142.0,-0.40,0.70,24.51,AR,NTT,CERAMICS
39,INFY,2275.0,-17.44,52.0,H-LC,9.13,321353.0,8465.0,163215.0,-1.11,2.71,50.79,54.87,3.0,0.05,2.34,9.53,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,62.0,H-SC,12.49,127251.0,-15228.0,34752.0,1.09,-10.69,27.31,13.70,163.0,-0.44,0.93,50.19,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.48,59.0,M-SC,4.71,88130.0,-12647.0,12744.0,0.41,-12.55,14.46,0.09,245.0,-0.99,0.64,15.78,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.29,45.0,H-SC,2.15,145475.0,-30428.0,72447.0,0.05,-17.30,49.80,23.89,149.0,-0.42,1.06,25.64,OX40N,NTT,CERAMICS
44,KANSAINER,340.0,-67.61,59.0,H-SC,1.81,222966.0,-46701.0,83033.0,0.08,-17.32,37.24,13.47,138.0,-0.56,1.62,12.77,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-20.01,53.0,H-MC,7.85,108310.0,-23525.0,65755.0,-0.37,-17.84,60.71,32.03,98.0,-0.36,0.79,21.32,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,52.0,H-LC,13.11,292782.0,-53174.0,177016.0,-0.60,-15.37,60.46,35.80,1.0,-0.30,2.13,3.72,X40,BTT,IT
39,INFY,2275.00,-17.44,52.0,H-LC,9.13,321353.0,8465.0,163215.0,-1.11,2.71,50.79,54.87,3.0,0.05,2.34,9.53,X40,BTT,IT
41,ITC,452.00,-37.50,54.0,H-LC,2.90,202226.0,2088.0,18807.0,-0.05,1.04,9.30,10.44,4.0,0.11,1.47,6.66,X40,NTT,FMCG
83,VBL,671.64,-16.43,38.0,H-LC,5.81,300881.0,-14961.0,127634.0,0.38,-4.74,42.42,35.67,5.0,-0.12,2.19,8.46,X40N,ATH,FMCG
1,ABB,7934.00,-39.18,59.0,H-LC,11.55,254016.0,-7603.0,126805.0,0.83,-2.91,49.92,45.57,7.0,-0.06,1.85,8.09,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,65.0,M-MC,6.94,228251.0,3289.0,164364.0,0.22,1.46,72.01,74.53,192.0,0.02,1.66,34.19,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.82,188960.0,24290.0,43669.0,0.25,14.75,23.11,41.28,89.0,0.56,1.37,23.91,X40N,ATH,AC
84,VOLTAS,1918.49,0.74,58.0,H-MC,2.83,211485.0,19743.0,76283.0,0.45,10.30,36.07,50.08,99.0,0.26,1.54,17.79,XY25,ATH,AC
24,DABUR,735.00,-3.30,56.0,H-MC,6.37,210308.0,13280.0,76342.0,0.07,6.74,36.30,45.49,102.0,0.17,1.53,20.66,XY24,BTT,FMCG
18,CAMS,5211.76,-5.38,50.0,H-SC,4.68,107587.0,5583.0,38344.0,-0.43,5.47,35.64,43.06,122.0,0.15,0.78,23.53,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.00,72.0,H-MC,5.05,115158.0,14654.0,15984.0,-0.47,14.58,13.88,30.49,79.0,0.92,0.84,51.71,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.16,55.0,M-SC,1.05,101198.0,9700.0,69058.0,-0.16,10.60,68.24,86.07,223.0,0.14,0.74,46.96,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.91,51.0,H-SC,12.23,137238.0,13902.0,119864.0,-0.43,11.27,87.34,108.45,119.0,0.12,1.00,36.14,AR,ATH,MISC
86,WIPRO,420.00,-11.52,52.0,M-LC,5.70,155459.0,4514.0,104935.0,-0.48,2.99,67.50,72.51,53.0,0.04,1.13,9.19,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.78,182084.0,2276.0,110143.0,0.89,1.27,60.49,62.52,88.0,0.02,1.32,35.78,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.79,82554.0,-19656.0,113049.0,1.02,-19.23,136.94,91.37,208.0,-0.17,0.60,34.01,XY25,NTT,FINANCE
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85440.0,-28109.0,68164.0,-1.66,-24.75,79.78,35.27,268.0,-0.41,0.62,109.12,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.85,43.0,H-SC,13.72,87591.0,-13372.0,107903.0,0.32,-13.24,123.19,93.63,148.0,-0.12,0.64,30.46,SR,ATH,CHEMICALS
48,MASFIN,398.61,-15.09,60.0,H-SC,12.85,96285.0,-1695.0,23301.0,-0.56,-1.73,24.20,22.05,152.0,-0.07,0.70,39.96,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.00,72.0,H-MC,5.05,115158.0,14654.0,15984.0,-0.47,14.58,13.88,30.49,79.0,0.92,0.84,51.71,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85440.0,-28109.0,68164.0,-1.66,-24.75,79.78,35.27,268.0,-0.41,0.62,109.12,XR,NTT,HOTELS
75,TITAGARH,1548.00,2.50,72.0,H-SC,4.89,168096.0,-23919.0,102808.0,3.56,-12.46,61.16,41.08,158.0,-0.23,1.22,42.44,XY24,NTT,ENGINEERING
8,ATULAUTO,844.00,3872.97,65.0,M-SC,7.20,123829.0,-23539.0,75350.0,8.27,-15.97,60.85,35.16,236.0,-0.31,0.90,26.48,XY24,NTT,AUTO
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.79,82554.0,-19656.0,113049.0,1.02,-19.23,136.94,91.37,208.0,-0.17,0.60,34.01,XY25,NTT,FINANCE
49,NATIONALUM,247.44,-42.00,72.0,H-MC,5.05,115158.0,14654.0,15984.0,-0.47,14.58,13.88,30.49,79.0,0.92,0.84,51.71,MH,ATH,METALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.43
2,50,73.81


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.49
LC    34.46
MC    23.05
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40      15.23
X40N     12.94
XR       10.05
XY25      9.79
AR        8.61
OX40N     8.22
X200      1.74
MH        1.73
X5K       1.46
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.43
H-LC    26.21
H-MC    20.09
M-SC    13.52
M-LC     7.18
M-MC     2.65
L-SC     1.54
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.36,-14.47,79.55
FMCG,11.84,-4.04,40.44
FINANCE,9.63,-16.16,61.66
BANKS,7.81,-11.73,63.18
MISC,6.04,-14.21,79.41
PAINTS,5.85,-14.76,40.44
ELECTRICAL,5.61,-6.51,65.21
AC,3.62,2.71,30.23
INSURANCE,3.40,-7.48,44.34


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2860325.0,22
AR,1312879.0,9
XR,1229928.0,13
X40,873167.0,10
X40N,630279.0,10
OX40N,519307.0,10
XY25,406752.0,7
SR,246301.0,2
X5K,116168.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3341070.0,28
H-LC,1390026.0,15
M-SC,1386439.0,17
H-MC,1189862.0,15
M-LC,389832.0,5
M-MC,313653.0,2
L-SC,249244.0,3
L-MC,60267.0,1
L-LC,37566.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1033078.0      6
M-SC       XY24       800900.0      7
H-SC       AR         784790.0      5
           XR         673499.0      6
H-LC       X40        587120.0      5
H-MC       XY24       488869.0      4
H-LC       AR         365121.0      2
M-MC       XY24       313653.0      2
H-SC       X40N       297501.0      4
           SR         246301.0      2
           OX40N      244434.0      4
H-LC       X40N       227090.0      4
H-MC       X40        217483.0      4
           XY25       185940.0      2
L-SC       XR         165194.0      2
M-SC       AR         162968.0      2
M-LC       XY24       158630.0      2
M-SC       OX40N      125068.0      4
M-LC       X5K        116168.0      1
M-SC       XR         115890.0      2
           XY25       113049.0      1
H-MC       XR         110143.0      1
           X40N       105688.0      2
M-LC       XR         104935.0      1
H-LC       X200        85402.0      1
L-SC       OX40N       84050.0      1
M-SC       X40         68564.0      1
H-MC       OX40N       65755.0      1
H-LC       XY24        65195.0      1
H-SC       MH          61467.0      1
L-MC       XR          60267.0      1
H-LC       XY25        60098.0      2
L-LC       XY25        37566.0      1
H-MC       MH          15984.0      1
M-LC       XY25        10099.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
